# NIFTY 200 – EMA20 + GAP Short Strategy

## Strategy Logic

1. Previous Close < EMA20 (Bearish Trend)
2. Gap % within defined range
3. Short at Open
4. Exit at Close
5. Position sizing using fixed capital

Output:
- Trades sheet
- Date-wise Summary
- Cumulative PnL

In [9]:
!pip install pandas yfinance requests xlsxwriter

# IMPORT LIBRARIES

In [18]:
import os
import requests
import pandas as pd
import yfinance as yf
from io import StringIO

# USER INPUT SECTION

In [19]:
# ================= USER INPUT =================

START_DATE = "2023-01-01"
END_DATE   = "2025-12-31"

DATA_FOLDER = "NIFTY200_DATA"
OUTPUT_FILE = "NIFTY200_EMA20_GAP_OUTPUT.xlsx"

EMA_PERIOD = 20
CAPITAL = 100000

# Gap %
GAP_MIN1, GAP_MAX1 = 0.3,1.2
GAP_MIN2, GAP_MAX2 = 2.1,2.6

print("Inputs Loaded Successfully")

Inputs Loaded Successfully


# DOWNLOAD NIFTY 200 STOCK LIST

In [20]:
def get_nifty200_symbols():

    url = "https://archives.nseindia.com/content/indices/ind_nifty200list.csv"
    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(url, headers=headers)
    data = StringIO(response.text)

    df = pd.read_csv(data)
    symbols = df["Symbol"].dropna().tolist()

    print("Total Stocks:", len(symbols))
    return symbols

symbols = get_nifty200_symbols()

Total Stocks: 200


# DOWNLOAD HISTORICAL DATA

In [21]:
os.makedirs(DATA_FOLDER, exist_ok=True)

for symbol in symbols:
    try:
        yahoo_symbol = symbol + ".NS"

        df = yf.download(
            yahoo_symbol,
            start=START_DATE,
            end=END_DATE,
            interval="1d",
            progress=False
        )

        if df.empty:
            continue

        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        df.reset_index(inplace=True)

        df.to_csv(os.path.join(DATA_FOLDER, f"{symbol}.csv"), index=False)

        print("Saved:", symbol)

    except Exception as e:
        print("Error:", symbol, e)

print("Download Completed")

Saved: 360ONE
Saved: ABB
Saved: ACC
Saved: APLAPOLLO
Saved: AUBANK
Saved: ADANIENSOL
Saved: ADANIENT
Saved: ADANIGREEN
Saved: ADANIPORTS
Saved: ADANIPOWER
Saved: ATGL
Saved: ABCAPITAL
Saved: ALKEM
Saved: AMBUJACEM
Saved: APOLLOHOSP
Saved: ASHOKLEY
Saved: ASIANPAINT
Saved: ASTRAL
Saved: AUROPHARMA
Saved: DMART
Saved: AXISBANK
Saved: BSE
Saved: BAJAJ-AUTO
Saved: BAJFINANCE
Saved: BAJAJFINSV
Saved: BAJAJHLDNG
Saved: BAJAJHFL
Saved: BANKBARODA
Saved: BANKINDIA
Saved: BDL
Saved: BEL
Saved: BHARATFORG
Saved: BHEL
Saved: BPCL
Saved: BHARTIARTL
Saved: BHARTIHEXA
Saved: BIOCON
Saved: BLUESTARCO
Saved: BOSCHLTD
Saved: BRITANNIA
Saved: CGPOWER
Saved: CANBK
Saved: CHOLAFIN
Saved: CIPLA
Saved: COALINDIA
Saved: COCHINSHIP
Saved: COFORGE
Saved: COLPAL
Saved: CONCOR
Saved: COROMANDEL
Saved: CUMMINSIND
Saved: DLF
Saved: DABUR
Saved: DIVISLAB
Saved: DIXON
Saved: DRREDDY
Saved: EICHERMOT
Saved: ETERNAL
Saved: EXIDEIND
Saved: NYKAA
Saved: FEDERALBNK
Saved: FORTIS
Saved: GAIL
Saved: GMRAIRPORT
Saved: GLENM

# APPLY STRATEGY

In [22]:
all_trades = []

for file in os.listdir(DATA_FOLDER):

    if not file.endswith(".csv"):
        continue

    file_path = os.path.join(DATA_FOLDER, file)
    symbol = file.replace(".csv", "")

    df = pd.read_csv(file_path)

    if df.empty:
        continue

    # Remove duplicate headers
    df = df[df["Date"] != "Date"]

    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.sort_values("Date").reset_index(drop=True)

    # Convert to numeric
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df = df.dropna(subset=["Close"])

    # Indicators
    df["EMA20"] = df["Close"].ewm(span=EMA_PERIOD, adjust=False).mean()

    df["Prev_Close"] = df["Close"].shift(1)
    df["Gap"] = df["Open"] - df["Prev_Close"]
    df["Gap_pct"] = (df["Gap"] / df["Prev_Close"]) * 100

    # Conditions
    below_ema = df["Close"].shift(1) < df["EMA20"].shift(1)

    gap_condition = (
        ((df["Gap_pct"] >= GAP_MIN1) & (df["Gap_pct"] <= GAP_MAX1)) |
        ((df["Gap_pct"] >= GAP_MIN2) & (df["Gap_pct"] <= GAP_MAX2))
    )

    df = df[below_ema & gap_condition]

    if df.empty:
        continue

    # Short Trade
    df["OC_Return"] = df["Open"] - df["Close"]
    df["Qty"] = (CAPITAL / df["Open"]).round(0)
    df["PnL"] = df["OC_Return"] * df["Qty"]

    df["Symbol"] = symbol
    df["Capital"] = CAPITAL

    all_trades.append(df)

print("Strategy Applied")

Strategy Applied


# CREATE FINAL TRADE DATAFRAME

In [23]:
if not all_trades:
    print("No Trades Found")
else:
    trades_df = pd.concat(all_trades).sort_values("Date").reset_index(drop=True)

trades_df.head()

,Date,Close,High,Low,Open,Volume,EMA20,Prev_Close,Gap,Gap_pct,OC_Return,Qty,PnL,Symbol,Capital
0,2023-01-04,1661.951294,1699.019102,1657.231781,1693.512963,435968,1697.217202,1688.252686,5.260278,0.311581,31.561669,59.0,1862.138499,GRASIM,100000
1,2023-01-04,402.008759,409.259245,400.784018,408.573379,641742,409.581844,407.054657,1.518722,0.373100,6.564620,245.0,1608.331950,SONACOMS,100000
2,2023-01-04,1192.541748,1226.055634,1188.289519,1226.055634,1376606,1217.934984,1219.977539,6.078095,0.498214,33.513886,82.0,2748.138626,HAL,100000
3,2023-01-04,2400.388184,2421.494680,2391.822421,2414.017518,1672649,2418.150857,2406.019775,7.997742,0.332406,13.629334,41.0,558.802696,HINDUNILVR,100000
4,2023-01-04,2619.145020,2630.614376,2594.686196,2616.190920,53699,2624.861557,2606.242188,9.948733,0.381727,-2.954100,38.0,-112.255782,OFSS,100000


# SUMMARY REPORT

In [24]:
summary_df = (
    trades_df
    .groupby("Date")
    .agg(
        No_of_Stocks=("Symbol", "count"),
        Sum_of_PnL=("PnL", "sum")
    )
    .reset_index()
)

summary_df["Cumulative_PnL"] = summary_df["Sum_of_PnL"].cumsum()

summary_df.head()

,Date,No_of_Stocks,Sum_of_PnL,Cumulative_PnL
0,2023-01-04,16,19241.576896,19241.576896
1,2023-01-05,80,6532.359758,25773.936654
2,2023-01-06,38,52802.991626,78576.928280
3,2023-01-09,102,-41171.269926,37405.658354
4,2023-01-10,24,28928.584034,66334.242388


# SAVE OUTPUT

In [25]:
with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as writer:
    trades_df.to_excel(writer, sheet_name="Trades", index=False)
    summary_df.to_excel(writer, sheet_name="Date_Summary", index=False)

print("Output Saved:", OUTPUT_FILE)

Output Saved: NIFTY200_EMA20_GAP_OUTPUT.xlsx


                                             ##########  Thank You  ################